# 머신러닝 프로젝트 : 향수 추천시스템

 - 크롤링 사이트 : https://www.basenotes.net

In [3]:
import requests
from bs4 import BeautifulSoup as bs
import time

## 전체 item url 및 ID 데이터 크롤링

- item_url : item 상세페이지로 가는 url
- item_id : item 고유 id
- review : review 갯수

In [ ]:
alphas = ["9", "a", "b", "c", "d", "e", "f", "g", "h", 'i', "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]

item_df = pd.DataFrame()
for alpha in alphas:
    url = "https://www.basenotes.net/fragrancedirectory/?i={0}&p=1".format(alpha)
    res = requests.get(url=url)
    soup = bs(res.text, "html.parser")
    pages = len(soup.select("div.threadpagenav")[0].select("span"))
    time.sleep(1)
    for page in range(1, pages -1):
        url = "https://www.basenotes.net/fragrancedirectory/?i={0}&p={1}".format(alpha, page)
        res = requests.get(url=url)
        soup = bs(res.text, "html.parser")
        a = soup.findAll("table")[1].find_all("tr", {"valign":"top"})
        if len(a)>20:
            del a[20]
        if len(a)>40:
            del a[40]
        if len(a)>60:
            del a[60]
        item_urls = [x.select("a")[0].get("href") for x in a]
        item_ids = [x.select("a")[0].get("href").split("/")[3][:10] for x in a]
        b = [x.select("div") for x in a]
        reviews = [None if x == [] else int(x[0].text.strip().split(" ")[0]) for x in b]
        temp = pd.DataFrame({"item_url":item_urls, "item_id":item_ids, "review":reviews})
        item_df = pd.concat([item_df, temp], axis=0)

In [383]:
item_df = item_df.drop(index=0, axis=0)
item_df.to_csv("item_df.csv", index=False)

## 각 item 별 정보 크롤링

In [1]:
item_df = pd.read_csv("item_df.csv")

너무 인지도가 없거나 정보가 없는 향수는 제외하기 위해서 리뷰 수가 5 이상인 것들만 추출

In [13]:
item_df = item_df[item_df.review > 4]
item_df.reset_index(inplace=True, drop=True)

url을 이용하여 각 아이템별 상세 정보 끌어오기

In [ ]:
for idx, row in item_df.iterrows():
    time.sleep(1)
    result = requests.get(url=row.item_url)
    soup = bs(result.text, "html.parser")
    datas = soup.find("div", {"class":"dirwrap50"})
    try:
        item_df.loc[idx,"names"] = datas.find("h2").text.strip()
    except:
        item_df.loc[idx,"names"] = None

    notes_ls = []
    try :
        notes_ls += datas.find_all("div", {"class":"dirright50"})[0].text.split("\n")[5].strip().split(", ")
        notes_ls += datas.find_all("div", {"class":"dirright50"})[0].text.split("\n")[12].strip().split(", ")
        notes_ls += datas.find_all("div", {"class":"dirright50"})[0].text.split("\n")[19].strip().split(", ")
    except :
        try :
            notes_ls += soup.find_all("div", {"class":"notespyramid notespyramidb"})[0].text.split("\n")[0].split(", ")
        except:
    item_df.loc[idx, "notes"] = str(notes_ls)
    
    brands = datas.find("div", {"class":"dirleft-content50"}).find_all("td", {"colspan":"3"})
    for brand in brands:
        if brand.select_one("a") != None:
            href = brand.select_one("a").get("href")
            if "brand" in href:
                brand_name = brand.select_one("a").text
            else :
                brands = datas.find("div", {"class":"dirleft-content50"}).find_all("td", {"colspan":"4"})
                for brand in brands:
                    if brand.select_one("a") != None:
                        href = brand.select_one("a").get("href")
                        if "brand" in href:
                            brand_name = brand.select_one("a").text
                        else :  
                            brand_name = None
                
    item_df.loc[idx,"house"] = brand_name
    
    try:
        rate = datas.find("div", {"class":"dirleft-content50"}).select_one("meta").get("content")[:4]
    except:
        rate = datas.find_all("td",{"colspan":"3"})[1].text.strip()[:-1]
    item_df.loc[idx,"rating"] = rate    
    
    item_df.loc[idx, "gender"] = datas.find("td",{"colspan":"2"}).text

In [25]:
item_df.to_csv("item_df_final.csv", index=False)

In [1]:
pd.read_csv("item_df_final_final.csv")

,item_url,item_id,review,names,notes,house,rating,gender
0,https://www.basenotes.net/ID26125576.html,ID26125576,83.0,Insolence Eau de Toilette,"['Bergamot', 'Raspberry', 'Red berries', 'Lemo...",LVMH Moet Hennessy Louis Vuitton,3.41,Feminine
1,https://www.basenotes.net/ID26125684.html,ID26125684,6.0,Inspiration,"['Pink peppercorn', 'Pomegranate', 'Mirabelle ...",Procter & Gamble,3.52,Feminine
2,https://www.basenotes.net/ID26159251.html,ID26159251,5.0,Instant Crush,"['Citrus fruits', 'Saffron', 'Ginger', 'Morocc...",Procter & Gamble,3.86,Feminine
3,https://www.basenotes.net/ID26144138.html,ID26144138,13.0,Insurrection II Pure,"['Pineapple', 'Vanilla', 'Birch']",Procter & Gamble,3.79,Masculine
4,https://www.basenotes.net/ID26143347.html,ID26143347,10.0,Intense Black,"['Calabrian lemon', 'Italian bergamot', 'Atlas...",Parour,3.64,Masculine
...,...,...,...,...,...,...,...,...
5465,https://www.basenotes.net/ID10210418.html,ID10210418,13.0,Initial,"['Blackcurrant', 'Muguet', 'Tangerine', 'Rose'...",Inter Parfums,3.49,Feminine
5466,https://www.basenotes.net/ID10213063.html,ID10213063,10.0,Inouï,"['Galbanum', 'Peach', 'Juniper', 'Lemon', 'Gre...",Groupe Clarins,4.30,Feminine
5467,https://www.basenotes.net/ID26120837.html,ID26120837,68.0,Insensé,"['Blackcurrant', 'Mandarin', 'Basil', 'Magnoli...",LVMH Moet Hennessy Louis Vuitton,3.88,Masculine
5468,https://www.basenotes.net/ID26120838.html,ID26120838,38.0,Insensé Ultramarine,"['Water fruits', 'Galbanum', 'Cardamom', 'Mint...",LVMH Moet Hennessy Louis Vuitton,3.26,Masculine
